# Holdout tcn eval

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import holidays

# --- Hyperparameters ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EMBED_DIM = 8
HIDDEN_DIM = 64
N_LAYERS = 4
LR = 0.0005
WEIGHT_DECAY = 0.0001
DROPOUT = 0.2
EPOCHS = 100
PATIENCE = 8
BATCH_SIZE = 128
VAL_FRAC = 0.2
STRIDE = 1  # Or whatever stride you want

# --- Load data ---
df = pd.read_csv("bicikelj_train.csv")
meta = pd.read_csv("bicikelj_metadata.csv")
station_cols = df.columns[1:]

# Clean and fill
for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# --- Load weather ---
weather_df = pd.read_csv("weather_ljubljana.csv", skiprows=2)
weather_df = weather_df.rename(columns={
    'temperature_2m (°C)': 'temperature_2m',
    'precipitation (mm)': 'precipitation',
    'windspeed_10m (km/h)': 'windspeed_10m',
    'cloudcover (%)': 'cloudcover'
})
weather_df['time'] = pd.to_datetime(weather_df['time'])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
df_merged = pd.merge(df, weather_df, left_on='timestamp', right_on='time', how='left')
weather_features = ['temperature_2m', 'precipitation', 'windspeed_10m', 'cloudcover']
df_merged[weather_features] = df_merged[weather_features].ffill().bfill()
N = len(df_merged)

# --- Randomly select validation cut regions ---
BLOCK_SIZE = HISTORY_LEN + PRED_HORIZON
np.random.seed(42)
all_possible_starts = np.arange(0, N - BLOCK_SIZE + 1)
val_mask = np.zeros(N, dtype=bool)
val_starts = []
target_val_coverage = int(VAL_FRAC * N)
covered = 0

np.random.shuffle(all_possible_starts)
for start in all_possible_starts:
    if val_mask[start:start + BLOCK_SIZE].any():
        continue  # Skip if this region overlaps with any already taken
    val_mask[start:start + BLOCK_SIZE] = True
    val_starts.append(start)
    covered += BLOCK_SIZE
    if covered >= target_val_coverage:
        break

train_mask = ~val_mask

print(f"Validation coverage: {val_mask.sum()} ({val_mask.sum()/N:.3f})")
print(f"Train coverage:      {train_mask.sum()} ({train_mask.sum()/N:.3f})")
print(f"Number of val sequences: {len(val_starts)}")

# --- Strided window sample selection ---
def make_sample_indices(mask, history_len, pred_horizon, stride=1):
    N = len(mask)
    indices = []
    for i in range(history_len, N - pred_horizon + 1, stride):
        if mask[i - history_len:i + pred_horizon].all():
            indices.append(i)
    return indices

train_indices = make_sample_indices(train_mask, HISTORY_LEN, PRED_HORIZON, stride=STRIDE)
val_indices = make_sample_indices(val_mask, HISTORY_LEN, PRED_HORIZON, stride=1)

print(f"Train indices: {len(train_indices)} | Val indices: {len(val_indices)}")

# --- Normalize using only train region statistics ---
station_means = df_merged.loc[train_mask, station_cols].mean()
station_stds  = df_merged.loc[train_mask, station_cols].std().replace(0, 1)
weather_means = df_merged.loc[train_mask, weather_features].mean()
weather_stds  = df_merged.loc[train_mask, weather_features].std().replace(0, 1)

df_merged[station_cols] = (df_merged[station_cols] - station_means) / station_stds
df_merged[weather_features] = (df_merged[weather_features] - weather_means) / weather_stds

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class SharedTCNDataset(Dataset):
    def __init__(self, df, station_cols, neighbors, history_len, pred_horizon, weather_features, sample_indices):
        self.samples = []
        self.station_to_idx = {name: i for i, name in enumerate(station_cols)}
        timestamps = pd.to_datetime(df['timestamp'])
        hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
        hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
        dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
        dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
        month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
        month_cos = np.cos(2 * np.pi * timestamps.dt.month / 12)
        is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
        slo_holidays = holidays.Slovenia()
        is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)
        weather_array = df[weather_features].values
        time_feats = np.concatenate([
            np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                      month_sin, month_cos, is_weekend, is_holiday], axis=1),
            weather_array
        ], axis=1)
        bikes = df[station_cols].values.astype(np.float32)
        N = len(df)
        for s_name in station_cols:
            s_idx = self.station_to_idx[s_name]
            nn_idx = [self.station_to_idx[nn] for nn in neighbors[s_name]]
            series = bikes[:, [s_idx] + nn_idx]
            full_feats = np.concatenate([series, time_feats], axis=1)
            for i in sample_indices:
                x = full_feats[i - history_len:i]
                y = bikes[i:i + pred_horizon, s_idx]
                self.samples.append((x, y, s_idx))
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y, sid = self.samples[idx]
        return (torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(sid, dtype=torch.long))

# --- TCN Block ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)
        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)
        self.head = nn.Sequential(
            nn.Linear(num_channels[-1] + embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)
        return self.head(combined)

# --- Prepare datasets and loaders ---
train_dataset = SharedTCNDataset(df_merged, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON, weather_features, train_indices)
val_dataset   = SharedTCNDataset(df_merged, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON, weather_features, val_indices)

# --- Holdout logic: split val_dataset into val/holdout Subsets (50/50 random) ---
val_indices_list = np.arange(len(val_dataset))
np.random.seed(42)
np.random.shuffle(val_indices_list)
split_point = len(val_indices_list) // 2
val_subset = Subset(val_dataset, val_indices_list[:split_point])
holdout_subset = Subset(val_dataset, val_indices_list[split_point:])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)
holdout_loader = DataLoader(holdout_subset, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)

print(f"Train samples: {len(train_dataset)} | Val samples: {len(val_subset)} | Holdout samples: {len(holdout_subset)}")

# --- Training ---
model = TCN(input_size=1 + K_NEIGHBORS + (8 + len(weather_features)),
            output_size=PRED_HORIZON,
            num_channels=[HIDDEN_DIM] * N_LAYERS,
            kernel_size=3,
            dropout=DROPOUT,
            num_stations=len(station_cols),
            embed_dim=EMBED_DIM).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()
best_loss = float('inf')
best_state = None
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for xb, yb, sid in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(xb, sid), yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb, sid in val_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            val_loss += criterion(model(xb, sid), yb).item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}: Val Loss = {avg_val_loss:.4f}")
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping!")
            break

model.load_state_dict(best_state)
print("✅ Model trained.")

# --- Holdout evaluation ---
model.eval()
holdout_loss = 0.0
with torch.no_grad():
    for xb, yb, sid in holdout_loader:
        xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
        holdout_loss += criterion(model(xb, sid), yb).item()
avg_holdout_loss = holdout_loss / len(holdout_loader)
print(f"Holdout loss: {avg_holdout_loss:.4f}")

torch.save(model.state_dict(), "tcn_model_final_with_holdout.pt")
print("✅ Saved model to 'tcn_model_final_with_holdout.pt'")


Validation coverage: 4108 (0.201)
Train coverage:      16355 (0.799)
Number of val sequences: 79
Train indices: 12934 | Val indices: 79
Train samples: 1086456 | Val samples: 3318 | Holdout samples: 3318


Epoch 1/100: 100%|██████████| 4244/4244 [00:33<00:00, 125.56it/s]

Epoch 1: Train Loss = 0.3520


Epoch 1: Val Loss = 0.3188


Epoch 2/100: 100%|██████████| 4244/4244 [00:33<00:00, 125.02it/s]

...

Epoch 25: Train Loss = 0.2864


Epoch 25: Val Loss = 0.3008


Epoch 26/100: 100%|██████████| 4244/4244 [00:34<00:00, 123.22it/s]

Epoch 26: Train Loss = 0.2861


Epoch 26: Val Loss = 0.3033
Early stopping!
✅ Model trained.
Holdout loss: 0.3040
✅ Saved model to 'tcn_model_final_with_holdout.pt'


# MSE on holdout

In [ ]:
import numpy as np
import torch
from sklearn.metrics import mean_squared_error

# Ensure DEVICE, model, holdout_loader, station_means, station_stds are in scope
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for xb, yb, sid in holdout_loader:
        xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
        preds = model(xb, sid)
        preds = preds.cpu().numpy()
        yb = yb.cpu().numpy()
        sid = sid.cpu().numpy()
        for b in range(preds.shape[0]):
            s = sid[b]
            mean = station_means.iloc[s]
            std = station_stds.iloc[s]
            all_preds.append(preds[b] * std + mean)
            all_targets.append(yb[b] * std + mean)

all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)
holdout_loss_real = mean_squared_error(all_targets.flatten(), all_preds.flatten())
print(f"Holdout MSE (unnormalized, real units): {holdout_loss_real:.4f}")


Holdout MSE (unnormalized, real units): 9.4291
